In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
cd ~/demres

/Users/zurfarosa/demres


In [3]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import pandas as pd
import numpy as np
from datetime import date, timedelta

import demres
from demres.common.constants import entry_type
from demres.demins.constants import Study_Design as sd
from demres.common import codelists
from demres.common.helper_functions import *
from demres.demins.statistical_functions import *

/Users/zurfarosa/demres/env/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [4]:
pd.set_option('display.max_columns', None)
pd.options.display.max_rows = 500

In [5]:
window = '10_to_5' #options: '12_to_7','10_to_5','8_to_3'

In [6]:
# pt_features = pd.read_csv('data/pt_data/processed_data/pt_features_demins_'+subtype+'_'+window+'.csv',delimiter=',',parse_dates=['index_date','data_end','data_start'],infer_datetime_format=True )

In [7]:
pt_features = pd.read_csv('data/pt_data/processed_data/pt_features_demins_'+window+'.csv',delimiter=',',parse_dates=['index_date','data_end','data_start'],infer_datetime_format=True )



In [8]:
len(pt_features[pt_features['isCase']==True])

12879

In [9]:
pt_features.sample(5)

,patid,yob,pracid,female,index_date,isCase,final dementia medcode,data_end,data_start,matchid,age_at_index_date,insomnia,insomnia_no_hypnotics,stroke,heart_failure,mental_illness,sleep_apnoea,chronic_pulmonary_disease,epilepsy,hypnotics_100_pdds,age_at_index_date:65-69,age_at_index_date:70-74,age_at_index_date:75-79,age_at_index_date:80-84,age_at_index_date:85-89,age_at_index_date:90-99,age_at_index_date:above_99,hypnotic_pdds:00000,hypnotic_pdds:00001_10,hypnotic_pdds:00011_100,hypnotic_pdds:00101_1000,hypnotic_pdds:01001_10000,hypnotic_pdds:10000_and_above
18030,6598045,26,45,1,2008-03-06,True,1917.0,2008-09-26,1993-01-31,6033,82,0,0,0,0,1,0,0,0,0.140389,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
6382,5194441,34,441,0,2007-03-09,True,7664.0,2009-01-27,1995-11-30,60762,73,0,0,0,0,0,0,0,0,0.000000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
16674,6398345,22,345,1,2008-03-20,False,NaN,2013-05-07,1998-01-31,48281,86,0,0,0,0,0,0,0,0,0.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
14235,2208231,27,231,0,2008-09-02,False,NaN,2011-09-07,1997-01-16,68279,81,0,0,0,0,0,0,0,0,0.000000,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
20120,1349444,17,444,1,2001-10-04,True,1917.0,2007-06-21,1991-01-31,61386,84,0,0,0,0,1,0,0,0,0.000000,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [10]:
pt_features.columns

Index(['patid', 'yob', 'pracid', 'female', 'index_date', 'isCase',
       'final dementia medcode', 'data_end', 'data_start', 'matchid',
       'age_at_index_date', 'insomnia', 'insomnia_no_hypnotics', 'stroke',
       'heart_failure', 'mental_illness', 'sleep_apnoea',
       'chronic_pulmonary_disease', 'epilepsy', 'hypnotics_100_pdds',
       'age_at_index_date:65-69', 'age_at_index_date:70-74',
       'age_at_index_date:75-79', 'age_at_index_date:80-84',
       'age_at_index_date:85-89', 'age_at_index_date:90-99',
       'age_at_index_date:above_99', 'hypnotic_pdds:00000',
       'hypnotic_pdds:00001_10', 'hypnotic_pdds:00011_100',
       'hypnotic_pdds:00101_1000', 'hypnotic_pdds:01001_10000',
       'hypnotic_pdds:10000_and_above'],
      dtype='object')

In [11]:
# characteristics = [column for column in pt_features.columns if column not in [
#     'patid',  'yob', 'pracid', 'index_date', 'isCase',
#     'final dementia medcode', 'data_start', 'data_end', 'matchid']]

In [12]:
columns_for_inclusion = [
    'age_at_index_date',
    'female',
    'stroke',
    'heart_failure',
    'mental_illness',
    'sleep_apnoea', 
    'chronic_pulmonary_disease', 
    'epilepsy',
    'hypnotics_100_pdds',
    'insomnia',
    'age_at_index_date:65-69', 'age_at_index_date:70-74',
    'age_at_index_date:75-79', 'age_at_index_date:80-84',
    'age_at_index_date:85-89', 'age_at_index_date:90-99',
    'age_at_index_date:above_99', 
    'hypnotics_100_pdds','hypnotic_pdds:00000',
    'hypnotic_pdds:00001_10', 'hypnotic_pdds:00011_100',
    'hypnotic_pdds:00101_1000', 'hypnotic_pdds:01001_10000',
    'hypnotic_pdds:10000_and_above'
]

In [13]:
baselines = add_baseline_characteristics(columns_for_inclusion,pt_features)

In [14]:
len(pt_features[pt_features['isCase']==False]),len(pt_features[pt_features['isCase']==True])

(12879, 12879)

In [15]:
baselines[['Cases','Controls']]

,Cases,Controls
female,8552 (66.4%),8552 (66.4%)
stroke,1431 (11.1%),959 (7.4%)
heart_failure,764 (5.9%),618 (4.8%)
mental_illness,4766 (37.0%),3019 (23.4%)
sleep_apnoea,22 (0.2%),17 (0.1%)
chronic_pulmonary_disease,2944 (22.9%),2114 (16.4%)
epilepsy,239 (1.9%),140 (1.1%)
insomnia,926 (7.2%),580 (4.5%)
age_at_index_date:65-69,452 (3.5%),452 (3.5%)
age_at_index_date:70-74,1161 (9.0%),1161 (9.0%)
